**SMT solver and the Python API.**

In the last lab, we have been using the SMT solver in Z3 via the SMT-LIB format. We can also use the SMT solver in Z3 via the Python API. The basic operations work exactly as we have learnt in Lab 2 -- we simply use a different sort than `Bool`:

In [1]:
from z3 import *

x = Real('x')
y = Real('y')
z = Real('z')
i = Int('i')

def model(phi):
    
    # create a SAT instance
    s = Solver()
    s.add(phi)
    
    # return a satisfying assignment
    return s.model() if s.check() == sat else ["unsat"]

print(model(x > y))

# retrieve a Real from the model, as a floating point value
print(float(model(x > y)[x].as_string()))

# retrieve an Int from the model, as Python int
print(model(i >= 3)[i].as_long())

# quantifiers
print(model(ForAll(x, Exists(y, y > x))))

# at most 1 condition satisfied
print(model(ForAll([x, y], AtMost(x>y, y>x, x==y, 1))))
# at least 2 conditions satisfied
print(model(Exists([x, y], AtLeast(x>y, y>x, x==y, 2))))

# function symbols -- e.g., function from RxR to R
f = Function('f', RealSort(), RealSort(), RealSort())
print(model(ForAll([x,y], f(x,y) == f(y,x))))
print(model(Exists([x,y], f(x,y) != f(y,x))))

# declare a new sort:

s = DeclareSort('s')
s1 = Const('s1', s)
s2 = Const('s2', s)
s3 = Const('s3', s)
print(model([s1 != s2, Exists(s3, And(s3 != s1, s3 != s2))]))



[y = -1, x = 0]
0.0
3
[]
[]
['unsat']
[f = [else -> 0]]
[x!19 = 0,
 y!18 = 1,
 f = [(0, 1) -> 2, (1, 0) -> 3, else -> 2]]
[s2 = s!val!1, s1 = s!val!0, s3!20 = s!val!2]


**Exercise 1 [Sudoku].**
Solve a Sudoku puzzle using Z3. You have to fill the dots in a 9x9 grid with digits from '1' to '9' in such a way that:

* in each row, every number appears exactly once,
* in each column, every number appears exactly once,
* our grid consists of nine 3x3 squares -- in each of them, every number has to appear exactly once.


In [2]:
puzzle = [
"..53.....",
"8......2.",
".7..1.5..",
"4....53..",
".1..7...6",
"..32...8.",
".6.5....9",
"..4....3.",
".....97.."]

# Hint: the boolean expression Distinct(x,y,z,...) checks whether the given values are distinct

In [3]:
from __future__ import print_function

cells = {(i,j): Int('{}{}'.format(i,j)) for i in range(9)
                                        for j in range(9)}

formulas = []

for l in range(9):
    for r in range(9):
        # Istniejące komórki
        if puzzle[l][r] != '.':
            formulas.append(cells[(l,r)] == int(puzzle[l][r]))
        
        # Komórki w zakresie 1 - 9
        formulas.append(1 <= cells[(l,r)])
        formulas.append(cells[(l,r)] <= 9)
        
# Różne komórki w linii i kolumnach
for l in range(9):
    formulas.append(Distinct(*[cells[(l, r)] for r in range(9)]))
    formulas.append(Distinct(*[cells[(r, l)] for r in range(9)]))

# Różne komórki w każdym kwadraciku
for i in range(3):
    for j in range(3):
        square = [cells[(i*3+k, j*3+l)] for k in range(3) for l in range(3)]
        formulas.append(Distinct(*square))

m = model(formulas)
for l in range(9):
    for r in range(9):
        print(m[cells[(l, r)]], end=" ")
    print("")


1 4 5 3 2 7 6 9 8 
8 3 9 6 5 4 1 2 7 
6 7 2 9 1 8 5 4 3 
4 9 6 1 8 5 3 7 2 
2 1 8 4 7 3 9 5 6 
7 5 3 2 9 6 4 8 1 
3 6 7 5 4 2 8 1 9 
9 8 4 7 6 1 2 3 5 
5 2 1 8 3 9 7 6 4 


**Optimizer.** We can also use Z3 to find a solution which maximizes or minimizes the given goal.

In [4]:
x = Real('x')
y = Real('y')

def maximize(phi, f):
    opt = Optimize()
    opt.add(phi)
    opt.maximize(f)
    return opt.model() if opt.check() else ["unsat"]

print(maximize([x*2+y <= 100, x+y*3 <= 100], x+y))

[y = 20, x = 40]


### **Exercise 2 [Minimum Weight Dominating Set].**
In the Minimum Weighted Dominating Set problem, you are given a graph $(V,E)$ and a weight function $w:V \rightarrow \mathbb{N}$. A *dominating set* is a set $W \subseteq V$ such that every vertex $v \in V$ is in $W$ itself, or is adjacent to a vertex in $W$. You have to find a dominating set $W \subseteq V$ such that the total weight of all vertices in $V$ is minimal. 

* Construct a graph with 100 vertices with random weights from 1 to 10, and n random edges.
* Use Z3 to find the minimum weight dominating set.
* Do this for n=0,...400. Examine how the time needed to compute the minimum weight dominating set changes. (It changes not only with n -- if we are unlucky, we can get instances much harder than usual ones with the given n!)
* Does the problem become easier to solve for Z3 when we consider "simple" graphs (e.g., paths, trees, 5xK grids)? (All these graphs have low 'treewidth'. Many hard problems, including the dominating set problem, can be solved in linear time on graphs of treewidth $\leq k$ using dynamic programming.)

In [5]:
from random import randint

def simulation(v, n):
    weights = [randint(1, 10) for i in range(v)]
    edges = []
    
    for i in range(n):
        a = randint(0, v - 1)
        b = randint(0, v - 1)
        
        if a < b:
            (a, b) = (b, a)
        
        edges.append((a,b))
    
    # Formulation
    vertices = [Int('v{}'.format(i)) for i in range(v)]
    
    wsum = Int('wsum')
    formulas = []
    formulas += [Or(i == 0, i == 1) for i in vertices]
    formulas.append(wsum == Sum([ vertices[i] * weights[i] for i in range(v)]))
    formulas += [Or(vertices[a] == 1, vertices[b] == 1) for (a,b) in edges]
    
    opt = Optimize()
    opt.add(formulas)
    opt.minimize(wsum)
    m = opt.model() if opt.check() else ["unsat"]
    return m[wsum]


In [6]:
import time
import matplotlib.pyplot as plt

def measure_time(e):
    tb = time.time()
    simulation(100, e)
    te = time.time()
    return te - tb

def measure_run_time(e):
    retry = 10
    return sum([measure_time(e) for i in range(retry)]) / float(retry)

step = 10
plotpoints = [(i * step, measure_run_time(i * step)) for i in range(400 // step)]
plt.plot(*(zip(*plotpoints)))
plt.show()

KeyboardInterrupt: 

**Exercise 3 [Squaring the Square]**. You are given a list of integers `sizes`, and the number `n`. Each integer $i$ in `sizes` represents a square of dimensions $i \times i$.

Find out if it is possible to fit all the squares in a square of dimensions $n \times n$, without overlapping. (It takes Z3 about a minute to solve the case below.)


In [ ]:
sizes = [2, 4, 6, 7, 8, 9, 11, 15, 16, 17, 18, 19, 24, 25, 27, 29, 33, 35, 37, 42, 50]
n = 112

squares = [Int('s{}{}'.format(i, j)) for i in range(len(sizes)) for j in range(4)]

formulas = []

# All points are on plane
formulas += [0 <= i for i in squares]
formulas += [i <= n for i in squares]

# Squares are in correct canonical form
for i in range(len(sizes)):
    formulas.append(squares[i*4 + 0] < squares[i*4 + 2])
    formulas.append(squares[i*4 + 1] < squares[i*4 + 3])
    formulas.append(squares[i*4 + 2] - squares[i*4 + 0] == sizes[i])
    formulas.append(squares[i*4 + 3] - squares[i*4 + 1] == sizes[i])

def not_overlapping(s1, s2):
    pass

# Squares are not overlapping
for i in range(len(sizes)):
    for j in range(len(sizes)):
        if i >= j:
            continue
        
        formulas += not_overlapping(i, j)


Draw the result placement. Assign one of four colors to every square in such a way that adjacent squares are assigned different colors. We know this is possible from the Four Color Theorem.

In [ ]:
# imshow can be used to draw a 2D array of numbers easily:

import matplotlib.pyplot as plt

tab = [[1,1,2], [1,1,2], [3,4,4]]
plt.imshow(tab, cmap='magma', interpolation='nearest')
plt.show()